#### 协程

* async 修饰词声明异步函数
* 调用异步函数，得到一个协程对象（coroutine object)
* 执行
    - 通过 await 来调用：await 执行的效果，和 Python 正常执行是一样的，也就是说程序会阻塞在这里，进入被调用的协程函数，执行完毕返回后再继续
    - 通过 asyncio.create_task() 来创建任务
    - asyncio.run 来触发运行。asyncio.run 函数是 Python 3.7 之后才有的特性，可以让 Python 的协程接口变得非常简单，不用去理会事件循环怎么定义和怎么使用的问题
        + 一个非常好的编程规范是，asyncio.run(main()) 作为主程序的入口函数，在程序运行周期内，只调用一次 asyncio.run
* 协程和多线程的区别
    - 协程为单线程
    - 协程由用户决定，在哪些地方交出控制权，切换到下一个任务
* 写法更加简洁清晰，把 async / await 语法和 create_task 结合来用
* 写协程程序的时候，脑海中要有清晰的事件循环概念，知道程序在什么时候需要暂停、等待 I/O，什么时候需要一并执行到底
    - 知道一个任务的哪个环节会造成I/O阻塞，然后把这个环节的代码异步化处理，并且通过await来标识在任务的该环节中断该任务执行，从而去执行下一个事件循环任务
    - 充分利用CPU资源，避免CPU等待I/O造成CPU资源白白浪费。当之前任务的那个环节的I/O完成后，线程可以从await获取返回值，然后继续执行没有完成的剩余代码
* 问题
    - ` RuntimeWarning: coroutine 'main' was never awaited self.tb = tb RuntimeWarning: Enable tracemalloc to get the object allocation traceback`

In [1]:
import time

def crawl_page(url):
    print('crawling begin {}'.format(url))
    # 逻辑
    sleep_time = int(url.split('_')[-1])
    time.sleep(sleep_time)
    
    print('OK {}'.format(url))

def main(urls):
    for url in urls:
        crawl_page(url)

%time main(['url_1', 'url_2', 'url_3', 'url_4'])

crawling begin url_1
OK url_1
crawling begin url_2
OK url_2
crawling begin url_3
OK url_3
crawling begin url_4
OK url_4
CPU times: user 9.9 ms, sys: 884 µs, total: 10.8 ms
Wall time: 10 s


In [2]:
import asyncio

async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    for url in urls:
        await crawl_page(url)

%time asyncio.run(main(['url_1', 'url_2', 'url_3', 'url_4']))

/home/henry/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py:1413: RuntimeWarning: coroutine 'main' was never awaited
  self.tb = tb


RuntimeError: asyncio.run() cannot be called from a running event loop

In [5]:
# 改变为任务：运行总时长等于运行时间最长的爬虫
import asyncio

async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    print('OK {}'.format(url))

async def main(urls):
    tasks = [asyncio.create_task(crawl_page(url)) for url in urls]
    # 另一种写法
    await asyncio.gather(*tasks)
        
#     for task in tasks:
#         await task

%time asyncio.run(main(['url_1', 'url_2', 'url_3', 'url_4']))

RuntimeError: asyncio.run() cannot be called from a running event loop

In [10]:
# 协程基本
import asyncio

async def worker_1():
    print('worker_1 start')
    await asyncio.sleep(1)
    print('worker_1 done')

async def worker_2():
    print('worker_2 start')
    await asyncio.sleep(2)
    print('worker_2 done')

async def main():
    print('before await')
    await worker_1()
    print('awaited worker_1')
    await worker_2()
    print('awaited worker_2')

%time asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
# 限定协程任务运行时间
import asyncio

async def worker_1():
    await asyncio.sleep(1)
    return 1

async def worker_2():
    await asyncio.sleep(2)
    return 2 / 0

async def worker_3():
    await asyncio.sleep(3)
    return 3

async def main():
    task_1 = asyncio.create_task(worker_1())
    task_2 = asyncio.create_task(worker_2())
    task_3 = asyncio.create_task(worker_3())

    await asyncio.sleep(2)
    task_3.cancel()

    res = await asyncio.gather(task_1, task_2, task_3, return_exceptions=True)
    print(res)

%time asyncio.run(main())

In [6]:
# 生产者消费者模型
import asyncio
import random

async def consumer(queue, id):
    while True:
        val = await queue.get()
        print('{} get a val: {}'.format(id, val))
        await asyncio.sleep(1)

async def producer(queue, id):
    for i in range(5):
        val = random.randint(1, 10)
        await queue.put(val)
        print('{} put a val: {}'.format(id, val))
        await asyncio.sleep(1)

async def main():
    queue = asyncio.Queue()

    consumer_1 = asyncio.create_task(consumer(queue, 'consumer_1'))
    consumer_2 = asyncio.create_task(consumer(queue, 'consumer_2'))

    producer_1 = asyncio.create_task(producer(queue, 'producer_1'))
    producer_2 = asyncio.create_task(producer(queue, 'producer_2'))

    await asyncio.sleep(10)
    consumer_1.cancel()
    consumer_2.cancel()
    
    await asyncio.gather(consumer_1, consumer_2, producer_1, producer_2, return_exceptions=True)

%time asyncio.run(main())

/home/henry/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py:1413: RuntimeWarning: coroutine 'main' was never awaited
  self.tb = tb


RuntimeError: asyncio.run() cannot be called from a running event loop

In [4]:
# 获取豆瓣北京近期上映电影的名称、时间和海报
import requests
from bs4 import BeautifulSoup

def main():
    url = "https://movie.douban.com/cinema/later/beijing/"
    init_page = requests.get(url).content
    init_soup = BeautifulSoup(init_page, 'lxml')

    all_movies = init_soup.find('div', id="showing-soon")
    print(all_movies)
#     for each_movie in all_movies.find_all('div', class_="item"):
#         all_a_tag = each_movie.find_all('a')
#         all_li_tag = each_movie.find_all('li')

#         movie_name = all_a_tag[1].text
#         url_to_fetch = all_a_tag[1]['href']
#         movie_date = all_li_tag[0].text

#         response_item = requests.get(url_to_fetch).content
#         soup_item = BeautifulSoup(response_item, 'lxml')
#         img_tag = soup_item.find('img')

#         print('{} {} {}'.format(movie_name, movie_date, img_tag['src']))

%time main()

None
CPU times: user 31.5 ms, sys: 169 µs, total: 31.7 ms
Wall time: 191 ms


In [ ]:
# 协程实现回调
import asyncio

async def crawl_page(url):
    print('crawling {}'.format(url))
    sleep_time = int(url.split('_')[-1])
    await asyncio.sleep(sleep_time)
    return 'OK {}'.format(url)

async def main(urls):
    tasks = [asyncio.create_task(crawl_page(url)) for url in urls]
    for task in tasks:
        task.add_done_callback(lambda future: print('result: ', future.result()))
    await asyncio.gather(*tasks)

%time asyncio.run(main(['url_1', 'url_2', 'url_3', 'url_4']))